In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
from glob import glob
import time, gc
import cv2
from keras import backend as K
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization,Activation
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))


2025-08-18 08:17:51.898579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755505071.912748       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755505071.917104       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 08:17:51.933874: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (/usr/local/lib/python3.11/dist-packages/keras/api/preprocessing/image/__init__.py)

In [3]:
sample_submission = pd.read_csv("../input/plant-pathology-2020-fgvc7/sample_submission.csv")
test = pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")
train = pd.read_csv("../input/plant-pathology-2020-fgvc7/train.csv")


In [4]:
train.head()


,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,1,0
1,Train_1,1,0,0,0
2,Train_2,0,0,1,0
3,Train_3,1,0,0,0
4,Train_4,0,0,1,0


In [5]:
x = train['image_id']


In [6]:
img_size=80


In [7]:
train_image=[]
for name in train['image_id']:
    path='/kaggle/input/plant-pathology-2020-fgvc7/images/'+name+'.jpg'
    img=cv2.imread(path)
    image=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_AREA)
    train_image.append(image)


In [8]:
fig, ax = plt.subplots(1, 4, figsize=(15, 15))
for i in range(4):
    ax[i].set_axis_off()
    ax[i].imshow(train_image[i])


NameError: name 'plt' is not defined

In [9]:
test.head()


,image_id
0,Test_0
1,Test_1
2,Test_2
3,Test_3
4,Test_4


In [10]:
test_image=[]
for name in test['image_id']:
    path='/kaggle/input/plant-pathology-2020-fgvc7/images/'+name+'.jpg'
    img=cv2.imread(path)
    image=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_AREA)
    test_image.append(image)


In [11]:
fig, ax = plt.subplots(1, 4, figsize=(15, 15))
for i in range(4):
    ax[i].set_axis_off()
    ax[i].imshow(test_image[i])


NameError: name 'plt' is not defined

In [12]:
#from keras.preprocessing.image import img_to_array
X_Train = np.ndarray(shape=(len(train_image), img_size, img_size, 3),dtype = np.float32)
i=0
for image in train_image:
    #X_Train[i]=img_to_array(image)
    X_Train[i]=train_image[i]
    i=i+1
X_Train=X_Train/255
print('Train Shape: {}'.format(X_Train.shape))


Train Shape: (1638, 80, 80, 3)


In [13]:
X_Test = np.ndarray(shape=(len(test_image), img_size, img_size, 3),dtype = np.float32)
i=0
for image in test_image:
    #X_Test[i]=img_to_array(image)
    X_Test[i]=test_image[i]
    i=i+1
    
X_Test=X_Test/255
print('Test Shape: {}'.format(X_Test.shape))


Test Shape: (183, 80, 80, 3)


In [14]:
y = train.copy()
del y['image_id']
y.head()


,healthy,multiple_diseases,rust,scab
0,0,0,1,0
1,1,0,0,0
2,0,0,1,0
3,1,0,0,0
4,0,0,1,0


In [15]:
y_train = np.array(y.values)
print(y_train.shape,y_train[0])


(1638, 4) [0 0 1 0]


In [16]:
def construct_model():
    model = Sequential()
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', input_shape=(img_size, img_size, 3),
                activation='relu'))
    model.add(BatchNormalization()) # Normalize the activations of the previous layer at each batch
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
   
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(5, 5), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=256, kernel_size=(5, 5), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Flatten()) # Flatten the input
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    # Configure the learning process
    # The loss function is the objective that the model will try to minimize
    # For any classification problem, use accuracy metric
    optimizer = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    model.summary()
    return model


In [17]:
X_train, X_val, Y_train, Y_val = train_test_split(X_Train, y_train, test_size=0.2, random_state=42)


NameError: name 'train_test_split' is not defined

In [18]:
model = construct_model()
annealer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)
# Generates batches of image data with data augmentation
datagen = ImageDataGenerator(rotation_range=360, # Degree range for random rotations
                        width_shift_range=0.2, # Range for random horizontal shifts
                        height_shift_range=0.2, # Range for random vertical shifts
                        zoom_range=0.2, # Range for random zoom
                        horizontal_flip=True, # Randomly flip inputs horizontally
                        vertical_flip=True) # Randomly flip inputs vertically

datagen.fit(X_train)
# Fits the model on batches with real-time data augmentation
hist = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=32),
               steps_per_epoch=X_train.shape[0] // 32,
               epochs=80,
               verbose=1,
               callbacks=[annealer, checkpoint],
               validation_data=(X_val, Y_val))


NameError: name 'Sequential' is not defined

In [19]:
predict = model.predict(X_Test)
all_predict = np.ndarray(shape = (test.shape[0],4),dtype = np.float32)
for i in range(0,test.shape[0]):
    for j in range(0,4):
        if predict[i][j]==max(predict[i]):
            all_predict[i][j] = 1
        else:
            all_predict[i][j] = 0 


NameError: name 'model' is not defined

In [20]:
healthy = [y_test[0] for y_test in all_predict]
multiple_diseases = [y_test[1] for y_test in all_predict]
rust = [y_test[2] for y_test in all_predict]
scab = [y_test[3] for y_test in all_predict]


NameError: name 'all_predict' is not defined

In [21]:
df = {'image_id':test.image_id,'healthy':healthy,'multiple_diseases':multiple_diseases,'rust':rust,'scab':scab}


NameError: name 'healthy' is not defined

In [22]:
data = pd.DataFrame(df)
data.tail()


NameError: name 'df' is not defined

In [23]:
data.to_csv('submission.csv',index = False)


NameError: name 'data' is not defined